In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz

In [0]:
!pip install -q findspark
!pip install py4j
!pip install pyspark

     |████████████████████████████████| 204kB 9.5MB/s 
     |████████████████████████████████| 215.7MB 62kB/s 
     |████████████████████████████████| 204kB 53.0MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130388 sha256=4c547bef9b9820d8cc5c22688dd9402939547ef837c4aced48dc3885991fbead
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark
  Found existing installation: py4j 0.10.9
    Uninstalling py4j-0.10.9:
      Successfully uninstalled py4j-0.10.9


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
'''
import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7") #SPARK_HOME

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
'''

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Tools we need to connect to the Spark server, load our data,
# clean it and prepare itfrom pyspark import SparkContext

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import isnan, when, count, col# Set up constants

CSV_2019= "/content/gdrive/My Drive/Colab Notebooks/ApacheSpark/396352027_112019_597_airline_delay_causes.csv"

APP_NAME = "Flight Delays"
SPARK_URL = "local[*]"

RANDOM_SEED = 141109
TRAINING_DATA_RATIO = 0.7
RF_NUM_TREES = 8
RF_MAX_DEPTH = 4
RF_NUM_BINS = 32

In [0]:
# Connect to the Spark server

spark = SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()

# Load datasets

df = spark.read.options(header="true",inferschema = "true").csv(CSV_2019)

In [55]:
print(f"The shape is {df.count():d} rows by {len(df.columns):d} columns.")

The shape is 1762 rows by 22 columns.


In [56]:
null_counts = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c)for c in df.columns]).toPandas().to_dict(orient='records')
print(f"We have {sum(null_counts[0].values()):d} null values in this dataset.")

We have 1762 null values in this dataset.


In [57]:
df.show()

+----+------+-------+-----------------+-------+--------------------+-----------+---------+----------+-----------+------+-----------+----------------+-------------+------------+----------+--------------+-------------+---------+--------------+-------------------+----+
|year| month|carrier|     carrier_name|airport|        airport_name|arr_flights|arr_del15|carrier_ct| weather_ct|nas_ct|security_ct|late_aircraft_ct|arr_cancelled|arr_diverted| arr_delay| carrier_delay|weather_delay|nas_delay|security_delay|late_aircraft_delay|_c21|
+----+------+-------+-----------------+-------+--------------------+-----------+---------+----------+-----------+------+-----------+----------------+-------------+------------+----------+--------------+-------------+---------+--------------+-------------------+----+
|2019|    11|     9E|Endeavor Air Inc.|    ABE|Allentown/Bethleh...|       51.0|      3.0|       2.0|        0.0|   0.0|        0.0|             1.0|          0.0|         0.0|     127.0|          79

In [0]:
df = df.drop(df._c21)
df = df.na.drop()

In [59]:
print(f"The shape is {df.count():d} rows by {len(df.columns):d} columns.")

The shape is 1762 rows by 21 columns.


In [60]:
df.dtypes

[('year', 'int'),
 (' month', 'int'),
 ('carrier', 'string'),
 ('carrier_name', 'string'),
 ('airport', 'string'),
 ('airport_name', 'string'),
 ('arr_flights', 'double'),
 ('arr_del15', 'double'),
 ('carrier_ct', 'double'),
 (' weather_ct', 'double'),
 ('nas_ct', 'double'),
 ('security_ct', 'double'),
 ('late_aircraft_ct', 'double'),
 ('arr_cancelled', 'double'),
 ('arr_diverted', 'double'),
 (' arr_delay', 'double'),
 (' carrier_delay', 'double'),
 ('weather_delay', 'double'),
 ('nas_delay', 'double'),
 ('security_delay', 'double'),
 ('late_aircraft_delay', 'double')]

In [0]:
from pyspark.sql.functions import when   

df = df.withColumn('arr_cancelled', when(df['arr_cancelled']>0,1).otherwise(0))

In [62]:
df.select('arr_cancelled').distinct().rdd.map(lambda r: r[0]).collect()

[1, 0]

In [0]:
feature_cols = ['year', ' month', 'carrier_ct', ' weather_ct', 'nas_ct', 'security_ct', 'late_aircraft_ct']

In [0]:
df = VectorAssembler(inputCols=feature_cols, outputCol="features").transform(df)

In [71]:
df.select("arr_cancelled", "features").show(5)

+-------------+--------------------+
|arr_cancelled|            features|
+-------------+--------------------+
|            0|[2019.0,11.0,2.0,...|
|            0|[2019.0,11.0,6.56...|
|            0|[2019.0,11.0,6.02...|
|            1|[2019.0,11.0,0.64...|
|            0|[2019.0,11.0,78.6...|
+-------------+--------------------+
only showing top 5 rows



In [0]:
# Generate a labelIndexer
labelIndexer = StringIndexer(inputCol="arr_cancelled", outputCol="indexedLabel").fit(df)

# Generate the indexed feature vector
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(df)
    
# Split the data into training and tests sets
(trainingData, testData) = df.randomSplit([TRAINING_DATA_RATIO, 1 - TRAINING_DATA_RATIO])

# Train the RandomForest model
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=RF_NUM_TREES)

# Chain indexers and the forest models in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])

In [0]:
# Train model
model = pipeline.fit(trainingData)# Make predictions
predictions = model.transform(testData)

In [75]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Test Error = {(1.0 - accuracy):g}")
print(f"Accuracy = {accuracy:g}")

Test Error = 0.271375
Accuracy = 0.728625
